### CrewAI 설치

In [ ]:
%pip install -q crewai crewai-tools

### Agent 정의

In [ ]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# 보고서를 만드는 Task를 간단히 만들어보자
# 목차를 만드는 단계와 목차에 해당하는 본문에 해당하는 단계를 구분해서 두개의 하위 Task를 만들고 그것들을 수행하기 위한 에이전트를 만들어보자

#목차 설정 에이전트
outline_generator = Agent(
    role='Outline Generator', # 역할
    goal='Create structured outlines for articles on given topics. answer in Korean', # 목표, 한국어로 답변하게 answer in Korean 넣기
    llm = ChatOpenAI(model = "gpt-4o-mini", max_tokens = 1000), # 언어 모델 (목차는 어려운 작업을 수행하는건 아니기 때문에 가벼운 gpt-4o-mini 사용)
    backstory='You are an expert at organizing information and creating comprehensive outlines for various subjects.'
    # backstory는 목차를 설정하는데 있어서 내가 특정한 맥락을 집어넣고 싶을때 설정. 지금 설정한건 시스템 프롬프트에 조금 더 보조를 해주는 역할을 부여함 (너는 다양한 주제에 있어 포괄적인 목차를 작성하고 정보를 모을 수 있는 전문가다)
)
#본문 작성 에이전트 (목차를 기반해서 본문을 작성)
writer = Agent(
    role='Writer',
    goal='Create engaging content based on research. answer in Korean',
    llm = ChatOpenAI(model = "gpt-4o", max_tokens = 3000), # 본문작성은 문장력이 좋아야 되기 때문에 더 똑똑한 모델을 사용 max_tokens도 더 많이 설정
    backstory='You are a skilled writer who can transform complex information into readable content.'
)

### Task 정의

In [65]:
# Task 정의
# 지금 두 개의 하위 테스크가 있다. 어떤 목차를 만들지에 대한 아웃라인을 만드는 outline_task가 하나있고,
# 만들어진 목차를 기반으로해서 제대로된 보고서를 만들어 내는 writing_task가 하나있다.
# 필수 값 : description, agent, expected_output, (tools는 옵셔널한 값)

outline_task = Task(
    description='Create a detailed outline for an article about AI\'s impact on job markets', # AI가 취업 시장에 미치는 영향에 대한 기사의 목차를 써달라는 설명
    agent=outline_generator,
    expected_output="""A comprehensive outline covering the main aspects of AI\'s 
    influence on employment""" # 예상출력 (좀 더 포괄적인 목차를 작성하도록 이 형식을 써주고, AI가 일자리 시장에 미치는 중요한 영역에 대해서 커버할 수 있는 목차를 만들어 달라고 설명을 넣음)
)

writing_task = Task(
    description='Write an article about the findings from the research',
    agent=writer,
    expected_output='An engaging article discussing AI\'s influence on job markets'
)

### Crew 정의

In [69]:
# Crew 정의
ai_impact_crew = Crew(
    agents=[outline_generator, writer],
    tasks=[outline_task, writing_task],
    verbose=True
)

2024-08-31 11:47:22,663 - 26160 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [71]:
# Crew 실행
result = ai_impact_crew.kickoff() # kickoff()함수를 활용해 Crew를 실행시킬 수 있음

result
# CrewOutput으로 출력이 나온다.
# CrewOutput의 경우에는 마크다운으로 출력하는 경우가 많다.OpenAI이던 클루드 모델을 쓰던 얘네 학습이 보고서를 만들어 달라고하면 마크다운 형식으로 출력해주는게 많음
# 마크다운으로 출력하는 법은 맨 아래 참고

 [2024-08-31 11:47:33][DEBUG]: == Working Agent: Outline Generator
 [2024-08-31 11:47:33][INFO]: == Starting Task: Create a detailed outline for an article about AI's impact on job markets
 [2024-08-31 11:47:51][DEBUG]: == [Outline Generator] Task output: ### AI가 고용 시장에 미치는 영향에 대한 기사 상세 개요

#### 1. 인트로덕션: AI와 고용 시장 소개
   - **AI 정의 및 중요성**
     - 인공지능(AI)의 기본 개념 설명
     - AI 기술의 발전 배경 및 현재 상태
   - **고용 시장의 변화**
     - 전통적인 고용 시장의 구조
     - AI의 도입으로 인한 변화 개요

#### 2. 고용에 대한 AI의 긍정적 영향
   - **생산성 향상**
     - 자동화 및 효율성 증대
     - 반복적인 작업을 줄이고 창의적인 업무에 집중
   - **신규 직업 창출**
     - AI 관련 기술직 및 지원직의 증가
     - 새로운 산업 분야 및 비즈니스 모델 탄생
   - **업무 환경 개선**
     - 위험한 작업 환경의 개선
     - 인간과 AI의 협력으로 인한 작업 스트레스 감소

#### 3. 고용에 대한 AI의 부정적 영향
   - **일자리 감소**
     - 자동화로 인한 일자리 축소
     - 특정 직종의 소멸
   - **소득 불평등 심화**
     - 고숙련 노동자와 저숙련 노동자 간의 격차 확대
     - 기술 접근성의 차이로 인한 불평등
   - **직업 불안감 증가**
     - 지속적인 기술 변화에 대한 적응 부담
     - 고용 안정성의 감소

#### 4. AI 영향을 받는 산업별 사례 연구
   - **제조업**
     - 자동화 공정 및 로봇의 도입
     -

CrewOutput(raw='---\n\n### AI가 고용 시장에 미치는 영향에 대한 기사\n\n#### 1. 인트로덕션: AI와 고용 시장 소개\n\n**AI 정의 및 중요성**\n\n인공지능(AI)은 컴퓨터 시스템이 인간의 지능적인 행동을 모방할 수 있도록 하는 기술입니다. AI는 머신러닝과 딥러닝을 포함해 데이터 분석, 패턴 인식, 문제 해결 등을 통해 인간의 결정을 돕습니다. AI 기술은 지난 수십 년 동안 비약적으로 발전해왔으며, 현재는 거의 모든 산업 분야에서 활용되고 있습니다.\n\n**고용 시장의 변화**\n\n전통적인 고용 시장은 안정적인 직업과 명확한 역할 분담을 특징으로 합니다. 하지만 AI의 도입은 이러한 구조를 크게 변화시키고 있습니다. AI는 반복적이고 단순한 작업을 자동화하여 인간의 부담을 줄이는 동시에, 새로운 직무와 산업을 창출하고 있습니다.\n\n#### 2. 고용에 대한 AI의 긍정적 영향\n\n**생산성 향상**\n\nAI 기술은 자동화와 효율성을 극대화합니다. 반복적인 작업을 자동으로 수행함으로써 인간은 더 창의적이고 전략적인 업무에 집중할 수 있게 됩니다. 예를 들어, 제조업에서 로봇 공정 자동화는 생산성을 크게 향상시키고 있습니다.\n\n**신규 직업 창출**\n\nAI의 발전은 새로운 직업군을 만들어냅니다. AI 개발자, 데이터 분석가, 그리고 AI 시스템을 지원하는 직무들이 늘어나고 있습니다. 또한 AI를 기반으로 한 새로운 산업 분야와 비즈니스 모델이 탄생하고 있습니다. 예를 들어, 자율주행차 산업은 새로운 기술직과 지원직을 창출하고 있습니다.\n\n**업무 환경 개선**\n\n위험한 작업 환경에서는 AI가 인간을 대신해 작업을 수행함으로써 안전성을 높이고 있습니다. 또한 인간과 AI의 협력으로 업무 스트레스가 줄어드는 효과도 나타나고 있습니다. 예를 들어, AI 기반의 의료 진단 시스템은 의료진의 부담을 줄이고 진단의 정확성을 높이고 있습니다.\n\n#### 3. 고용에 대한 AI의 부정적 영향\n\n**일자리

### Process 정의

In [73]:
# Process 정의

from crewai.process import Process

# Process를 정의해주면 우리가 정의한 순서대로 에이전트가 작업을 수행하게 됨
ai_impact_crew = Crew(
    agents=[outline_generator, writer], # 첫 번째로 outline_generator가 수행되고, 두 번째로 writer가 수행되는 순서를 정의
    tasks=[outline_task, writing_task], # 첫 번째로 outline_generator가 되어 outline_task를 수행함
    verbose=True,
    Process=Process.sequential
)

2024-08-31 11:50:57,666 - 26160 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [75]:
# Crew 실행
result = ai_impact_crew.kickoff()

result

 [2024-08-31 11:50:58][DEBUG]: == Working Agent: Outline Generator
 [2024-08-31 11:50:58][INFO]: == Starting Task: Create a detailed outline for an article about AI's impact on job markets
 [2024-08-31 11:51:16][DEBUG]: == [Outline Generator] Task output: ### AI가 일자리 시장에 미치는 영향에 대한 기사 상세 윤곽

#### 서론
- **AI의 중요성**: 인공지능(AI)이 기술 혁신과 비즈니스 모델 변화에 어떻게 기여하고 있는지 개괄.
- **기사의 목적**: AI가 일자리 시장에 미치는 다양한 영향을 탐구하고, 이를 통해 독자들이 미래에 대비할 수 있도록 도움.

#### 1. 일자리 대체 (Job Displacement)
- **자동화와 일자리 감소**: 제조업, 서비스업 등 다양한 산업에서 자동화로 인해 사라지는 일자리.
  - 예: 자율 주행 차량으로 인한 운송업 일자리 감소.
- **기술 발전 속도**: AI 기술의 빠른 발전이 일자리에 미치는 충격의 속도와 강도.
- **취약 계층**: 저숙련 노동자들이 AI로 인해 일자리 상실의 주요 타겟이 되는 이유.

#### 2. 일자리 창출 (Job Creation)
- **새로운 직업군의 등장**: AI와 관련된 새로운 직업의 생성.
  - 예: 데이터 과학자, AI 연구원 등.
- **기술 보조 역할**: AI가 인간을 보조하는 역할을 통해 새로운 종류의 일자리 창출.
  - 예: AI 기반 헬스케어 어드바이저.
- **스타트업과 혁신**: AI 기술을 활용한 새로운 비즈니스 모델과 스타트업의 증가.

#### 3. 직업의 성격 변화 (Changes in Job Nature)
- **복합적 기술 요구**: AI와 협업하기 위해 필요한 복합적 기술과 다방면의 지식.
  - 예: AI 시스템 운영 및 유

CrewOutput(raw='### AI가 일자리 시장에 미치는 영향\n\n#### 서론\n인공지능(AI)은 현대 기술 혁신의 중심에 있으며, 이는 다양한 산업에서 비즈니스 모델의 변화를 주도하고 있습니다. 이번 기사에서는 AI가 일자리 시장에 미치는 다양한 영향을 탐구하고, 이를 통해 독자들이 미래에 대비할 수 있도록 돕고자 합니다.\n\n#### 1. 일자리 대체 (Job Displacement)\nAI와 자동화 기술은 제조업, 서비스업 등 여러 산업에서 일자리를 대체하고 있습니다. 예를 들어, 자율 주행 차량의 도입으로 운송업 분야의 일자리 감소가 예상됩니다. AI 기술의 빠른 발전은 일자리에 미치는 충격의 속도와 강도를 더욱 가속화하고 있으며, 저숙련 노동자들은 이로 인해 일자리 상실의 주요 타겟이 되고 있습니다.\n\n#### 2. 일자리 창출 (Job Creation)\nAI의 발전은 새로운 직업군의 등장을 촉진하고 있습니다. 데이터 과학자, AI 연구원 등 AI와 관련된 새로운 직업들이 생겨나고 있으며, AI가 인간을 보조하는 역할을 통해 새로운 종류의 일자리도 창출되고 있습니다. 예를 들어, AI 기반 헬스케어 어드바이저와 같은 직업이 이에 해당됩니다. 또한, AI 기술을 활용한 새로운 비즈니스 모델과 스타트업의 증가도 눈에 띕니다.\n\n#### 3. 직업의 성격 변화 (Changes in Job Nature)\nAI와 협업하기 위해 필요한 복합적 기술과 다방면의 지식이 요구되고 있습니다. AI 시스템의 운영 및 유지보수 능력과 같은 기술이 필요하며, 단순 반복 작업보다는 창의적이고 문제 해결에 집중하는 직업이 증가하고 있습니다. 또한, AI 기술을 활용한 원격 근무 및 유연 근무제의 확산으로 유연한 근무 환경이 조성되고 있습니다.\n\n#### 4. 미래에 필요한 기술 (Skills Required in the Future)\n미래에는 디지털 리터러시, 데이터 분석 능력, AI 윤리와 규제 이해, 지속 학습 능력 등이 중요해질 것입니다. 기본

In [ ]:
result

In [77]:
from IPython.display import display, Markdown  # IPython.display에 display, Markdown을 가져옴
display(Markdown(result.raw)) # display로 Markdown형식의 텍스트로 출력
                              # 이렇게 해주면 CrewOutput로 나온 출력 텍스트들이 마크다운 형식으로 출력됨

### AI가 일자리 시장에 미치는 영향

#### 서론
인공지능(AI)은 현대 기술 혁신의 중심에 있으며, 이는 다양한 산업에서 비즈니스 모델의 변화를 주도하고 있습니다. 이번 기사에서는 AI가 일자리 시장에 미치는 다양한 영향을 탐구하고, 이를 통해 독자들이 미래에 대비할 수 있도록 돕고자 합니다.

#### 1. 일자리 대체 (Job Displacement)
AI와 자동화 기술은 제조업, 서비스업 등 여러 산업에서 일자리를 대체하고 있습니다. 예를 들어, 자율 주행 차량의 도입으로 운송업 분야의 일자리 감소가 예상됩니다. AI 기술의 빠른 발전은 일자리에 미치는 충격의 속도와 강도를 더욱 가속화하고 있으며, 저숙련 노동자들은 이로 인해 일자리 상실의 주요 타겟이 되고 있습니다.

#### 2. 일자리 창출 (Job Creation)
AI의 발전은 새로운 직업군의 등장을 촉진하고 있습니다. 데이터 과학자, AI 연구원 등 AI와 관련된 새로운 직업들이 생겨나고 있으며, AI가 인간을 보조하는 역할을 통해 새로운 종류의 일자리도 창출되고 있습니다. 예를 들어, AI 기반 헬스케어 어드바이저와 같은 직업이 이에 해당됩니다. 또한, AI 기술을 활용한 새로운 비즈니스 모델과 스타트업의 증가도 눈에 띕니다.

#### 3. 직업의 성격 변화 (Changes in Job Nature)
AI와 협업하기 위해 필요한 복합적 기술과 다방면의 지식이 요구되고 있습니다. AI 시스템의 운영 및 유지보수 능력과 같은 기술이 필요하며, 단순 반복 작업보다는 창의적이고 문제 해결에 집중하는 직업이 증가하고 있습니다. 또한, AI 기술을 활용한 원격 근무 및 유연 근무제의 확산으로 유연한 근무 환경이 조성되고 있습니다.

#### 4. 미래에 필요한 기술 (Skills Required in the Future)
미래에는 디지털 리터러시, 데이터 분석 능력, AI 윤리와 규제 이해, 지속 학습 능력 등이 중요해질 것입니다. 기본적인 디지털 기술 이해와 활용 능력은 물론, 데이터 처리 및 분석 능력의 중요성이 커지고 있으며, AI 기술의 윤리적 문제와 규제에 대한 이해도 필요합니다. 급변하는 기술 환경에서 지속적으로 학습하고 적응하는 능력 역시 중요해질 것입니다.

#### 5. 경제적 영향 (Economic Implications)
AI는 생산성 향상과 경제 성장을 촉진할 수 있습니다. 그러나 기술 발전으로 인한 소득 불균형의 심화 가능성도 존재합니다. 기존 산업의 구조적 변화와 새로운 산업의 부상이 예상되며, 이로 인해 경제 전반에 걸쳐 다양한 변화가 일어날 것입니다.

#### 6. 부정적 영향 완화 방법 (Potential Solutions to Mitigate Negative Impacts)
정부와 기업은 AI로 인한 부정적 영향을 완화하기 위해 정책적 대응 방안을 마련해야 합니다. 예를 들어, 재교육 프로그램과 사회 안전망 강화를 통해 일자리 상실 문제를 해결할 수 있습니다. 또한, 미래 직업 환경에 맞춘 교육 시스템 개혁과 AI와 관련된 공공 인식 제고 및 사회적 논의 활성화가 필요합니다.

#### 결론
AI가 일자리 시장에 미치는 영향은 매우 다양합니다. 생산성 향상과 새로운 일자리 창출의 긍정적 측면이 있는 반면, 일자리 감소와 소득 불균형의 부정적 측면도 존재합니다. 이에 따라 개인과 사회는 AI 시대에 대비하여 필요한 기술을 습득하고, 지속적으로 학습하는 자세를 유지해야 합니다. 정부와 기업은 이러한 변화를 수용하고, 정책적 대응을 통해 사회적 안정성을 도모해야 할 것입니다.